In [2]:
import nltk
nltk.download('stopwords')
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from lightgbm import LGBMClassifier
from nltk.corpus import stopwords
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
import urllib3

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Os dados utilizados de noticias são do remositório [Fake.br-Corpus](https://github.com/roneysco/Fake.br-Corpu). São vários arquivos de texto com as notícias separados em diretórios diferentes, marcados como 'fake' e 'true' disponibilizados no GitHub. Exemplo do conteúdo de um arquivo:

In [3]:
http = urllib3.PoolManager() # PoolManager
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning) # Desabilitando avisos
url = "https://raw.githubusercontent.com/roneysco/Fake.br-Corpus/master/full_texts/fake/%d.txt" % 1 # URL da notícia
r = http.request('GET', url) # Copiando notícia
print(r.data.decode('utf-8')) # Print com o texto decodificado

Kátia Abreu diz que vai colocar sua expulsão em uma moldura, mas não para de reclamar.	

A senadora Kátia Abreu (sem partido-TO) disse que sua expulsão do PMDB foi resultado de uma ação da cúpula atual da legenda que, segundo ela, é oportunista.

“Amanhã eu vou botar numa moldura dourada a minha expulsão, porque das mãos de onde veio, é um atestado de boa conduta para o meu currículo. Essas pessoas que me expulsaram não servem ao país. Eles se servem do país em seus benefícios próprios”, disse Kátia Abreu.

Ué, mas se a expulsão é algo tão bom para seu currículo, por que tanta choradeira, Kátia?

Sabemos o motivo. Provavelmente Kátia não tem valor para o PT, partido que já deveria tê-la absorvido. Ao que parece o PT gostava de Kátia somente se ela ficasse entrincheirada dentro do PMDB.

Ou seja, isso é se rebaixar demais. Resta a Kátia ficar chorando as pitangas por todos os cantos.

Em tempo: até o momento o PT não cadastrou Kátia Abreu em suas fileiras. Que situação patética para a e

É criado um dataframe do pandas.

In [0]:
df = pd.DataFrame(columns = ['noticia', 'label'])

Adicionando as notícias ao dataframe (são 3600 notícias fake e 3600 notícias reais):

In [0]:
for i in range(1, 3601):
  url = "https://raw.githubusercontent.com/roneysco/Fake.br-Corpus/master/full_texts/fake/%d.txt" % i # URL da notícia
  r = http.request('GET', url) # Copiando notícia
  noticia = pd.DataFrame([[r.data.decode('utf-8'), 'fake']], columns = ['noticia', 'label'])
  df = df.append(noticia, ignore_index=True)

for i in range(1, 3601):
  url = "https://raw.githubusercontent.com/roneysco/Fake.br-Corpus/master/full_texts/true/%d.txt" % i # URL da notícia
  r = http.request('GET', url) # Copiando notícia
  noticia = pd.DataFrame([[r.data.decode('utf-8'), 'true']], columns = ['noticia', 'label'])
  df = df.append(noticia, ignore_index=True)

Início do data-frame

In [6]:
df.head()

,noticia,label
0,Kátia Abreu diz que vai colocar sua expulsão e...,fake
1,Blog esquerdista dá a entender que reclamar de...,fake
2,"Alckmin diz que por ele PSDB desembarca, mas...",fake
3,Cara de pau não tem limites: Zé Celso aciona M...,fake
4,Temer resolve o problema de Luislinda: liberd...,fake


Final do dataframe

In [7]:
df.tail()

,noticia,label
7195,Ficou longe das notícias no fim de ano? Veja o...,true
7196,A nova denúncia contra o ex-presidente Luiz I...,true
7197,Como a Bahia virou uma potência mundial do mar...,true
7198,"Alvo da Lava Jato, Bendine tinha passagem comp...",true
7199,Chefs convidados do Encontro Mundial das Cidad...,true


In [0]:
df2 = pd.read_csv('https://raw.githubusercontent.com/ViniciusNunes0/SIRENE-news/master/noticias-sirene.csv', sep=';')
df2 = df2[['noticia', 'classificacao']]
df2 = df2.rename(columns={"noticia": "noticia", "classificacao": "label"})
df2['label'] = df2['label'].replace({0: 'true', 1: 'fake'})
df2.head()
df = df.append(df2, ignore_index=True)

In [0]:
x_train,x_test,y_train,y_test=train_test_split(df['noticia'], df['label'], test_size=0.2, random_state=42)

In [0]:
tfidf_vectorizer = TfidfVectorizer(stop_words=stopwords.words('portuguese'),
                                analyzer='word',
                                ngram_range=(1, 1),
                                lowercase=True,
                                use_idf=True)

In [0]:
tfidf_train = tfidf_vectorizer.fit_transform(x_train) 
tfidf_test = tfidf_vectorizer.transform(x_test)

In [27]:
pac = PassiveAggressiveClassifier(max_iter=50, random_state=0)
pac.fit(tfidf_train,y_train)

y_pred = pac.predict(tfidf_test)
score = accuracy_score(y_test,y_pred)
print(f'Accuracy: {round(score*100,2)}%')

Accuracy: 96.69%


In [28]:
rf = RandomForestClassifier(random_state=0)
rf.fit(tfidf_train,y_train)

y_pred = rf.predict(tfidf_test)
score = accuracy_score(y_test,y_pred)
print(f'Accuracy: {round(score*100,2)}%')

Accuracy: 93.93%


In [41]:
lgbm = LGBMClassifier(learning_rate=0.1,
                      num_leaves=128,
                      min_child_samples=100,
                      ubsample=0.96,
                      colsample_bytree=0.28,
                      random_state=0,
                      subsample_freq=1,
                      n_estimators=100)
lgbm.fit(tfidf_train,y_train)

y_pred = lgbm.predict(tfidf_test)
score = accuracy_score(y_test,y_pred)
print(f'Accuracy: {round(score*100,2)}%')

Accuracy: 96.4%


In [30]:
!pip install scikit-optimize
from skopt import dummy_minimize

     |████████████████████████████████| 81kB 2.5MB/s 


In [0]:
def treinar_modelo(params):
    learning_rate = params[0]
    num_leaves = params[1]
    min_child_samples = params[2]
    subsample = params[3]
    colsample_bytree = params[4]
    
    print(params, '\n')
    
    mdl = LGBMClassifier(learning_rate=learning_rate,
                         num_leaves=num_leaves,
                         min_child_samples=min_child_samples,
                         subsample=subsample,
                         colsample_bytree=colsample_bytree,
                         random_state=0,
                         subsample_freq=1,
                         n_estimators=100)
    mdl.fit(tfidf_train,y_train)
    
    y_pred = mdl.predict(tfidf_test)
    
    return -accuracy_score(y_test,y_pred)

In [32]:
space = [(1e-3, 1e-1, 'log-uniform'), #learning rate
         (2, 128), # num_leaves
         (1, 100), # min_child_samples
         (0.05, 1.0), # subsample
         (0.1, 1.0)] # colsample bytree

resultado = dummy_minimize(treinar_modelo, space, random_state=1, verbose=1, n_calls=30)

Iteration No: 1 started. Evaluating function at random point.
[0.09871192514273254, 74, 10, 0.3372159440002478, 0.23208030173540176] 

Iteration No: 1 ended. Evaluation done at random point.
Time taken: 24.9603
Function value obtained: -0.9539
Current minimum: -0.9539
Iteration No: 2 started. Evaluating function at random point.
[0.001529949829431263, 78, 72, 0.3782826906908954, 0.457090726807603] 

Iteration No: 2 ended. Evaluation done at random point.
Time taken: 6.4905
Function value obtained: -0.9221
Current minimum: -0.9539
Iteration No: 3 started. Evaluating function at random point.
[0.01195730942971637, 128, 19, 0.5483207515942279, 0.49910760440160107] 

Iteration No: 3 ended. Evaluation done at random point.
Time taken: 37.0118
Function value obtained: -0.9338
Current minimum: -0.9539
Iteration No: 4 started. Evaluating function at random point.
[0.0028784217488024557, 16, 51, 0.9182639233502714, 0.5114843271882895] 

Iteration No: 4 ended. Evaluation done at random point.
Ti

In [35]:
resultado.x

[0.09871192514273254, 74, 10, 0.3372159440002478, 0.23208030173540176]

In [0]:
from skopt import gp_minimize

In [37]:
resultados_gp = gp_minimize(treinar_modelo, space, random_state=1, verbose=1, n_calls=50, n_random_starts=10)

Iteration No: 1 started. Evaluating function at random point.
[0.09871192514273254, 120, 14, 0.9990884895579377, 0.3124800792567785] 

Iteration No: 1 ended. Evaluation done at random point.
Time taken: 49.8891
Function value obtained: -0.9598
Current minimum: -0.9598
Iteration No: 2 started. Evaluating function at random point.
[0.006210998932353835, 51, 67, 0.9387621172657304, 0.8616798250174156] 

Iteration No: 2 ended. Evaluation done at random point.
Time taken: 17.2649
Function value obtained: -0.9121
Current minimum: -0.9598
Iteration No: 3 started. Evaluating function at random point.
[0.004232013397179603, 68, 45, 0.2680983530433343, 0.5809725180523154] 

Iteration No: 3 ended. Evaluation done at random point.
Time taken: 8.6664
Function value obtained: -0.9200
Current minimum: -0.9598
Iteration No: 4 started. Evaluating function at random point.
[0.0672858974212934, 60, 44, 0.9421713999524447, 0.8005503127028804] 

Iteration No: 4 ended. Evaluation done at random point.
Time 

In [39]:
resultados_gp.x

[0.1, 128, 100, 0.9562060254291818, 0.2826514395863122]

In [0]:
[0.1, 23, 51, 0.8039348617451548, 0.34702698317511055] 